<br><br>
<font color='FireBrick' size='7'><b> Double/Debiased Machine Learning</b></font>

<font color='FireBrick' size='5'><b>using IPUMS data</b></font>

In [1]:
import pandas as pd

main_data_df = pd.read_feather("/Users/dadmehr/R/main_data.feather")

print(f"Number of rows: {main_data_df.shape[0]}")
print(f"Number of columns: {main_data_df.shape[1]}")

Number of rows: 43424806
Number of columns: 54


In [3]:
data = data.sample(frac=0.01)  # Sampling 1% of the dataset due to GitHub's 25MB file size limit for uploads.

In [2]:
data = main_data_df[main_data_df['YEAR'] == 2021]
data = data[['INCTOT', 'SEX', 'EDUC', 'EDUCD', 'RACE', 'AGE','EMPSTAT', 'MARST']]

data.to_csv('IPUMS_2021_data.csv', index=False)

In [4]:
data

,INCTOT,SEX,EDUC,EDUCD,RACE,AGE,EMPSTAT,MARST
40411062,15000.0,2,2,23,8,53,3,2
42278278,100000.0,2,10,101,1,52,1,1
40433269,38000.0,2,8,81,1,39,3,1
40233437,132900.0,2,11,116,1,73,1,4
42664738,60000.0,1,6,63,1,43,1,1
...,...,...,...,...,...,...,...,...
40460287,9999999.0,1,0,1,6,1,0,6
42456835,22800.0,1,10,101,6,62,3,4
40906088,9999999.0,1,2,26,1,14,0,6
42583174,0.0,2,11,116,1,66,3,1


In [ ]:
#!pip install statsmodels econml

In [5]:
import statsmodels.api as sm

# Prepare the variables
X = data[['SEX']]  # Independent variables
y = data['INCTOT']  # Dependent variable

# Add a constant to the independent variables (for the intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print the summary of the regression model
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                 INCTOT   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     25.04
Date:                Tue, 23 Apr 2024   Prob (F-statistic):           5.64e-07
Time:                        13:20:16   Log-Likelihood:            -5.3734e+05
No. Observations:               32526   AIC:                         1.075e+06
Df Residuals:                   32524   BIC:                         1.075e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.91e+06   6.39e+04     29.891      0.0

In [6]:
import statsmodels.api as sm

# Prepare the variables
X = data[['SEX', 'AGE']]  # Independent variables
y = data['INCTOT']  # Dependent variable

# Add a constant to the independent variables (for the intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print the summary of the regression model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 INCTOT   R-squared:                       0.403
Model:                            OLS   Adj. R-squared:                  0.403
Method:                 Least Squares   F-statistic:                 1.098e+04
Date:                Tue, 23 Apr 2024   Prob (F-statistic):               0.00
Time:                        13:20:16   Log-Likelihood:            -5.2896e+05
No. Observations:               32526   AIC:                         1.058e+06
Df Residuals:                   32523   BIC:                         1.058e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.651e+06   5.55e+04    101.886      0.0

<font color='FireBrick' size='7'><b>Frisch–Waugh–Lovell (FWL) theorem</b></font>

In [7]:
import pandas as pd
import statsmodels.api as sm


# Define the variables
y = data['INCTOT']  # Income
x1 = data['SEX']  # Sex
x2 = data['AGE']  # Age

# Residualize y and x1 with respect to x2
y_resid = sm.OLS(y, x2).fit().resid
x1_resid = sm.OLS(x1, x2).fit().resid

# Regress y_resid on x1_resid
fwl_regression = sm.OLS(y_resid, x1_resid).fit()

# Print the FWL coefficient
print(fwl_regression.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.339
Model:                            OLS   Adj. R-squared (uncentered):              0.339
Method:                 Least Squares   F-statistic:                          1.667e+04
Date:                Tue, 23 Apr 2024   Prob (F-statistic):                        0.00
Time:                        13:20:16   Log-Likelihood:                     -5.3347e+05
No. Observations:               32526   AIC:                                  1.067e+06
Df Residuals:                   32525   BIC:                                  1.067e+06
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

<font color='FireBrick' size='7'><b>Why different coefficients</b></font>

The Frisch-Waugh-Lovell (FWL) theorem holds under certain assumptions, and there are cases where these assumptions are not met, making the theorem inapplicable or invalid. Here are some scenarios where the FWL theorem does not hold:

- **Non-linear relationships**: The FWL theorem assumes a linear relationship between the variables. If the relationships are non-linear, the theorem does not apply.
- **Non-constant variance**: The theorem assumes constant variance of the error terms. If the variance is not constant, the FWL theorem is not applicable.
- **Correlated errors**: The FWL theorem assumes independent and identically distributed error terms. If the errors are correlated, the theorem does not hold.
- **Multicollinearity**: If the independent variables are highly correlated, the FWL theorem may not be applicable due to multicollinearity issues.
- **Omitted variable bias**: If there are omitted variables that are correlated with both the independent and dependent variables, the FWL theorem may not hold.
- **Measurement error**: If there are measurement errors in the independent variables, the FWL theorem may not be applicable.
- **Time-series data**: The FWL theorem is typically applied to cross-sectional data. If the data is time-series, additional considerations and modifications are necessary.

Generalized linear models: The FWL theorem is specific to linear regression models. If the relationship is modeled using a generalized linear model (e.g., logistic regression), **the theorem does NOT apply**.
It's essential to carefully evaluate the assumptions and limitations of the FWL theorem before applying it to your data.

In [8]:
import doubleml as dml
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Load your data (replace 'your_data.csv' with your actual file name)

# Get the column names for the outcome, treatment, and control(s)
y_col = 'INCTOT'  # Outcome variable
d_col = 'SEX'  # Treatment variable
x_cols = ['AGE']  # Control variables

# Create the dataset for DoubleML
data_dml = dml.DoubleMLData(data, y_col=y_col, d_cols=[d_col], x_cols=x_cols)

# Initialize the machine learning algorithms for DML
ml_y = RandomForestRegressor()  # For the outcome variable
ml_d = RandomForestRegressor()  # For the treatment variable

# Initialize and fit the DoubleMLPLR model
dml_plr = dml.DoubleMLPLR(data_dml, ml_y, ml_d, n_folds=10)
dml_plr.fit()

# Retrieve the summary DataFrame
summary = dml_plr.summary

# Check the available column names
print(summary.columns)

# Extract the coefficient for the treatment variable (SEX)
sex_coef = summary['coef'].loc[d_col]

# Output the coefficient for SEX
print(f"The estimated coefficient for SEX is: {sex_coef}")

Index(['coef', 'std err', 't', 'P>|t|', '2.5 %', '97.5 %'], dtype='object')
The estimated coefficient for SEX is: -17907.976085269598


The estimated coefficient suggests that the treatment (SEX) has a significant negative effect on the outcome variable (INCTOT). The exact interpretation of the coefficient value depends on the context and the units of the variables. For example, if INCTOT represents income, the coefficient suggests that the treatment (SEX) is associated with a decrease in income of approximately 17974.03 units.

In [9]:
summary['coef']

SEX   -17907.976085
Name: coef, dtype: float64

i.e. women have 17K less in income than men 